In [4]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

[Learn the Basics](intro.html) \|\|
[Quickstart](quickstart_tutorial.html) \|\|
[Tensors](tensorqs_tutorial.html) \|\| [Datasets &
DataLoaders](data_tutorial.html) \|\|
[Transforms](transforms_tutorial.html) \|\| **Build Model** \|\|
[Autograd](autogradqs_tutorial.html) \|\|
[Optimization](optimization_tutorial.html) \|\| [Save & Load
Model](saveloadrun_tutorial.html)

Build the Neural Network
========================

Neural networks comprise of layers/modules that perform operations on
data. The [torch.nn](https://pytorch.org/docs/stable/nn.html) namespace
provides all the building blocks you need to build your own neural
network. Every module in PyTorch subclasses the
[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
A neural network is a module itself that consists of other modules
(layers). This nested structure allows for building and managing complex
architectures easily.

In the following sections, we\'ll build a neural network to classify
images in the FashionMNIST dataset.


In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [5]:
!pip install --upgrade torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Get Device for Training
=======================

We want to be able to train our model on an
[accelerator](https://pytorch.org/docs/stable/torch.html#accelerators)
such as CUDA, MPS, MTIA, or XPU. If the current accelerator is
available, we will use it. Otherwise, we use the CPU.


In [3]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


Define the Class
================

We define our neural network by subclassing `nn.Module`, and initialize
the neural network layers in `__init__`. Every `nn.Module` subclass
implements the operations on input data in the `forward` method.


In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__() #必须调用 super().__init__() 是Python中调用父类初始化方法的关键操作，在PyTorch模型中有重要作用：
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            #层间结构：Linear(784→512) → ReLU → Linear(512→512) → ReLU
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits #输出为原始logits（未做softmax归一化）

We create an instance of `NeuralNetwork`, and move it to the `device`,
and print its structure.


In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model\'s
`forward`, along with some [background
operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866).
Do not call `model.forward()` directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0
corresponding to each output of 10 raw predicted values for each class,
and dim=1 corresponding to the individual values of each output. We get
the prediction probabilities by passing it through an instance of the
`nn.Softmax` module.


In [14]:
# 1. 生成随机输入数据
X = torch.rand(1, 28, 28, device=device)
# 创建一个形状为 (1,28,28) 的随机张量
# 表示一个批次的单个样本（MNIST风格的手写数字图像）
# device=device 表示将数据放在GPU/CPU设备上（与模型保持一致）

# 2. 模型推理得到原始输出
logits = model(X)
print(logits)
# 将输入数据传入神经网络模型
# logits 是未经归一化的原始预测值（每个类别对应的分数）

# 3. 转换为概率分布
pred_probab = nn.Softmax(dim=1)(logits)
print(pred_probab)
# 使用Softmax函数将logits转换为概率值
# dim=1 表示在类别维度进行归一化（假设输出形状为 [batch_size, num_classes]）
# 得到的pred_probab每个元素值在0-1之间，且同一行元素和为1

# 4. 获取预测结果
y_pred = pred_probab.argmax(1)
print(y_pred)
# 沿着维度1（类别维度）取最大值的索引
# 即选择概率最高的类别作为预测结果

# 5. 输出预测类别
print(f"Predicted class: {y_pred}")
# 输出结果格式示例：Predicted class: tensor([3])

tensor([[ 0.0062, -0.1019,  0.0146,  0.0132, -0.0177, -0.0316,  0.0267, -0.0239,
          0.0860,  0.0579]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)
tensor([0], device='cuda:0')
Predicted class: tensor([0], device='cuda:0')


------------------------------------------------------------------------


Model Layers
============

Let\'s break down the layers in the FashionMNIST model. To illustrate
it, we will take a sample minibatch of 3 images of size 28x28 and see
what happens to it as we pass it through the network.


In [18]:
input_image = torch.rand(3,28,28)
print(input_image.size)
#
print(input_image.size())

<built-in method size of Tensor object at 0x79554bd0df10>
torch.Size([3, 28, 28])


nn.Flatten
==========

We initialize the
[nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)
layer to convert each 2D 28x28 image into a contiguous array of 784
pixel values ( the minibatch dimension (at dim=0) is maintained).


In [19]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


nn.Linear
=========

The [linear
layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)
is a module that applies a linear transformation on the input using its
stored weights and biases.


In [20]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


nn.ReLU
=======

Non-linear activations are what create the complex mappings between the
model\'s inputs and outputs. They are applied after linear
transformations to introduce *nonlinearity*, helping neural networks
learn a wide variety of phenomena.

In this model, we use
[nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)
between our linear layers, but there\'s other activations to introduce
non-linearity in your model.


In [21]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.0051,  0.3543, -0.1237, -0.3317,  0.4811,  0.1254, -0.0880,  0.1331,
          0.6933, -0.2526,  0.5322,  0.3192,  0.0801, -0.4189,  0.2974, -0.5786,
          0.1114, -0.2556, -0.1355, -0.4292],
        [-0.6311,  0.4086, -0.2492, -0.1642,  0.3516,  0.2442, -0.0951, -0.3333,
          0.5001, -0.5065,  0.5591,  0.1821,  0.2854, -0.5396,  0.1742, -0.2742,
         -0.0049,  0.0220,  0.0744, -0.2247],
        [-0.2711, -0.0200, -0.2543, -0.3028,  0.3010, -0.2033, -0.1105, -0.0131,
          0.3737, -0.2942,  0.3299,  0.1635,  0.2744, -0.5084,  0.0652, -0.3281,
         -0.0941,  0.1969,  0.0199, -0.5422]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0051, 0.3543, 0.0000, 0.0000, 0.4811, 0.1254, 0.0000, 0.1331, 0.6933,
         0.0000, 0.5322, 0.3192, 0.0801, 0.0000, 0.2974, 0.0000, 0.1114, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.4086, 0.0000, 0.0000, 0.3516, 0.2442, 0.0000, 0.0000, 0.5001,
         0.0000, 0.5591, 0.1821, 0.2854, 0.0000, 0.17

nn.Sequential
=============

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)
is an ordered container of modules. The data is passed through all the
modules in the same order as defined. You can use sequential containers
to put together a quick network like `seq_modules`.


In [29]:
# 1. 创建输入数据
input_image = torch.rand(3,28,28)  # 生成形状为 [3,28,28] 的随机张量
                                   # 表示3个样本（图像），每个尺寸28x28像素
print(input_image.size())          # 输出：torch.Size([3, 28, 28])

# 2. 展平层操作
flatten = nn.Flatten()             # 创建展平层
flat_image = flatten(input_image)  # 将3D图像数据压平为2D
print(flat_image.size())           # 输出：torch.Size([3, 784])
                                   # 784 = 28x28（保留批次维度3）

# 3. 第一个全连接层
layer1 = nn.Linear(in_features=28*28, out_features=20)  # 创建线性层
hidden1 = layer1(flat_image)        # 执行线性变换
print(hidden1.size())               # 输出：torch.Size([3, 20] 每个样本转换为20维特征向量

# 4. ReLU激活函数
print(f"Before ReLU: {hidden1}\n\n")  # 显示激活前的原始值（可能有负数）
hidden1 = nn.ReLU()(hidden1)         # 应用ReLU函数（负数归零，正数保留）
print(f"After ReLU: {hidden1}")      # 显示激活后的值（全非负数）

# 5. 构建顺序模型 nn.Sequential
seq_modules = nn.Sequential(        # 创建顺序连接的神经网络
    flatten,                        # 1. 展平层
    layer1,         # 2. 第一个线性层
    nn.ReLU(),                      # 3. 激活函数
    nn.Linear(20,10)    # 4. 第二个线性层（输出10个类别分数）
)

# 6. 完整前向传播
input_image = torch.rand(3,28,28)   # 新输入（3个样本）
logits = seq_modules(input_image)   # 依次通过所有层
print(logits)                       # 输出形状为 [3,10] 的原始分数

# 7. Softmax概率转换
softmax = nn.Softmax(dim=1)         # 沿类别维度（dim=1）归一化
pred_probab = softmax(logits)       # 转换为概率分布
print(pred_probab)                  # 输出形状 [3,10]，每行和为1

torch.Size([3, 28, 28])
torch.Size([3, 784])
torch.Size([3, 20])
Before ReLU: tensor([[ 0.2506,  0.2862,  0.1746,  0.1698, -0.0621, -0.1257, -0.4932,  0.2838,
          0.5925, -0.3333, -0.1066, -0.0807,  0.0396, -0.5153, -0.0239, -0.4931,
         -0.4577, -0.4494, -0.4042, -0.5688],
        [ 0.2265,  0.2871,  0.2454,  0.0946, -0.1340, -0.0591, -0.4525, -0.1974,
          0.5135, -0.1394, -0.3606, -0.2426, -0.2600, -0.3155, -0.0384, -0.1588,
         -0.3818, -0.1798, -0.0999, -0.9599],
        [ 0.3670,  0.1982,  0.0610,  0.0270, -0.2409, -0.0382, -0.2978,  0.3405,
          0.5168,  0.1965,  0.0194,  0.0575, -0.1721, -0.2072,  0.1944,  0.0873,
         -0.2800, -0.2366, -0.1668, -0.5350]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.2506, 0.2862, 0.1746, 0.1698, 0.0000, 0.0000, 0.0000, 0.2838, 0.5925,
         0.0000, 0.0000, 0.0000, 0.0396, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.2265, 0.2871, 0.2454, 0.0946, 0.0000, 0.0000, 0.0000, 0.00

nn.Softmax
==========

The last linear layer of the neural network returns [logits]{.title-ref}
- raw values in \[-infty, infty\] - which are passed to the
[nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html)
module. The logits are scaled to values \[0, 1\] representing the
model\'s predicted probabilities for each class. `dim` parameter
indicates the dimension along which the values must sum to 1.


In [30]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)


tensor([[0.0939, 0.0810, 0.1236, 0.1285, 0.0976, 0.0778, 0.0863, 0.0816, 0.1325,
         0.0973],
        [0.0947, 0.0792, 0.1275, 0.1208, 0.0960, 0.0826, 0.0863, 0.0836, 0.1381,
         0.0913],
        [0.0896, 0.0830, 0.1201, 0.1305, 0.0976, 0.0823, 0.0917, 0.0816, 0.1251,
         0.0986]], grad_fn=<SoftmaxBackward0>)


Model Parameters
================

Many layers inside a neural network are *parameterized*, i.e. have
associated weights and biases that are optimized during training.
Subclassing `nn.Module` automatically tracks all fields defined inside
your model object, and makes all parameters accessible using your
model\'s `parameters()` or `named_parameters()` methods.

In this example, we iterate over each parameter, and print its size and
a preview of its values.


In [31]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0061,  0.0061,  0.0288,  ...,  0.0338, -0.0100, -0.0082],
        [ 0.0217, -0.0075,  0.0347,  ..., -0.0105,  0.0230, -0.0063]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0111, -0.0180], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0149,  0.0197,  0.0397,  ...,  0.0302,  0.0438, -0.0409],
        [ 0.0385,  0.0192, -0.0418,  ...,  0.0123,  0.0043,  0.0272]],
       device='cuda:0', grad_fn=<Sl

In [32]:
print(len(model.linear_relu_stack[0].bias))

512


------------------------------------------------------------------------


Further Reading
===============

-   [torch.nn API](https://pytorch.org/docs/stable/nn.html)
